# Efficient settings finder -
For efficienctly finding settings for lambda (temperature control) and current controls for matching of Bob's and Alice's lasers for MDI-QKD in the macro-frequency range (higher than 500MHz which is the sensitivity range for the FPGA). The assumption is that either Bob's of Alice's laser frequency is held constant while the other's parameters are tuned to match the frequency of the first one.

Note: This code implementation is based on assumption of linearity of response of frequency of the laser with respect to parameters (lambda and current). This assumption is based on the preliminary data collected on 24th May, 2023 for characterizing the laser freq. response wrt both parameters. 
In case linearity assumption does not hold, then there needs to a better determination of response function and more sofisticated algorithms (gradient descent etc.) would be required. 

Note 2: If liniarity holds for power of the laser as well wrt these parameter changes, the same algorithm can be used to find the effiecient parameter guess for it as well. Then, overlapping regions of effieicnt parameter choice can be used to both frequency and power of the two lasers.

Note 3: This implementation assumes that the data is fed manually, but in case of automated frequncy readers, this algorith could be easily adopted for the automated version.

In [4]:
#imports
import numpy as np
from pprint import pprint
from itertools import permutations

In [5]:
def get_input_values(max_lambda,min_lambda, max_current, min_current):
    mm = float(input(f'Frequency value at min_lambda = {min_lambda}, min_current = {min_current}:'))
    Mm = float(input(f'Frequency value at max_lambda = {max_lambda}, min_current = {min_current}:'))
    mM = float(input(f'Frequency value at min_lambda = {min_lambda}, max_current = {max_current}:'))
    MM = float(input(f'Frequency value at max_lambda = {max_lambda}, max_current = {max_current}:'))
    
    return (mm,Mm,mM,MM)

In [6]:
def estimate_macro_frequency_parameters(match_freq, max_lambda, min_lambda, max_current, min_current):
    (mm,Mm,mM,MM) = get_input_values(max_lambda,min_lambda, max_current, min_current)
    
    p1 = np.array([min_lambda, min_current, mm])
    p2 = np.array([max_lambda, min_current, Mm])
    p3 = np.array([max_lambda, max_current, MM])
    p4 = np.array([min_lambda, max_current, mM]) 
    
    points = []
    
    for perm in permutations((p1,p2,p3,p4),2):
        if perm[0][2]<=match_freq<=perm[1][2]:
            lengthx = abs(perm[0][0] - perm[1][0])
            lengthy = abs(perm[0][1] - perm[1][1])
            
            if (lengthx**2+lengthy**2)<=max((max_lambda-min_lambda)**2,(max_current-min_current)**2): 
                slope = (match_freq-perm[0][2])/(perm[1][2]-perm[0][2])
                xp = perm[0][0]+slope*(perm[1][0]-perm[0][0])
                yp = perm[0][1]+slope*(perm[1][1]-perm[0][1])
                points.append(np.array([xp,yp,match_freq]))
                
    if len(points)<=0:
        raise ValueError("Matching frequency is outside the parameter boundaries. It is suggested to change them.")
    
    return sum(points)/len(points)


# Example - 
match_freq = 1550.093

max_lambda = 500
min_lambda = -500
max_current = 1600
min_current = 500

estimate = estimate_macro_frequency_parameters(match_freq, max_lambda,min_lambda, max_current, min_current)

print(f"\nTry point (lambda = {estimate[0]}, current = {estimate[1]}) for matching frequency. \nIt is suggested to try nearby points of the suggested points as well, or use gradient descent near this point to find the optimal point.")

Frequency value at min_lambda = -500, min_current = 500:1550.121
Frequency value at max_lambda = 500, min_current = 500:1550.076
Frequency value at min_lambda = -500, max_current = 1600:1550.155
Frequency value at max_lambda = 500, max_current = 1600:1550.131

Try point (lambda = 311.1111111108304, current = 670.0000000003307) for matching frequency. 
It is suggested to try nearby points of the suggested points as well, or use gradient descent near this point to find the optimal point.
